# Lab 4 Answers:

### Exercise 1:

We first create the two RDDs (heathrowdata and wickairportData) from Lab 3:

In [ ]:
val heathrowFile = sc.textFile("file:///home/USER/bd-sp-2017/data/heathrowdata.txt")
val wickairportFile = sc.textFile("file:///home/USER/bd-sp-2017/data/wickairportdata.txt")

def isHeader(line: String): Boolean = {
  line.contains("yyyy")
}

case class TemperatureRecord(year: Long, month: Int, tmax: Float, tmin: Float, rain: Float)

def parse(line: String) = {
  val pieces = line.split('\t')
  val year = pieces(0).toLong
  val mm = pieces(1).toInt
  val tmax = pieces(2).toFloat
  val tmin = pieces(3).toFloat
  val rain = pieces(4).toFloat
  TemperatureRecord(year, mm, tmax, tmin, rain)
}

def isMissing(line: String): Boolean = {
  line.contains("---")
}

val heathrowData = heathrowFile.filter(x => !isHeader(x)).filter(x => !isMissing(x)).map(parse)
val wickairportData = wickairportFile.filter(x => !isHeader(x)).filter(x => !isMissing(x)).map(parse)

Note that USER will need to be replaced with your compute cluster username, userN. 

Now using the function we created to remove missing data we make a note of the year(s) and month(s) of the missing data records. First for Heathrow:

In [ ]:
heathrowFile.filter(x => isMissing(x)).foreach(println)

Nothing was printed so there is no missing data for Heathrow. Now for Wick:

In [ ]:
wickairportFile.filter(x => isMissing(x)).foreach(println)

The following output was obtained:

### Exercise 2:

Code to compute the average monthly max temperature for both airports (heathrowAverageTMax and wickAverageTMax) is included below:

In [ ]:
val heathrowAverageTmax = heathrowData.map(x => (x.year,x.tmax)).aggregateByKey((0.0,0.0))((acc,value) => (acc._1 + value, acc._2 + 1), (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)).mapValues(sumCount => 1.0 * sumCount._1 / sumCount._2).sortBy(_._1)
val wickairportAverageTmax = wickairportData.map(x => (x.year,x.tmax)).aggregateByKey((0.0,0.0))((acc,value) => (acc._1 + value, acc._2 + 1), (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)).mapValues(sumCount => 1.0 * sumCount._1 / sumCount._2).sortBy(_._1)

### Exercise 3:

Code to join the Tmax data from the heathrowData and wickairportData datasets is included below:

In [ ]:
val h_data = heathrowData.map(x => ((x.year,x.month),x.tmax))
val w_data = wickairportData.map(x => ((x.year,x.month),x.tmax))
val combinedData = h_data.join(w_data)

### Exercise 4:

Code to compute summary statistics for each monthly max temperature data is included below:

In [ ]:
val h_tmax_observations = combinedData.map(_._2._1).map(x => Vectors.dense(x.toDouble))
val w_tmax_observations = combinedData.map(_._2._2).map(x => Vectors.dense(x.toDouble))
val heathrow_summary: MultivariateStatisticalSummary = Statistics.colStats(h_tmax_observations)
val wick_summary: MultivariateStatisticalSummary = Statistics.colStats(w_tmax_observations)
println(heathrow_summary.mean)
println(heathrow_summary.variance)
println(wick_summary.mean)
println(wick_summary.variance)

The output of the last four print commands was:

### Exercise 5:

Code to compute the Pearson correlation coefficient for the two average max temperature datasets computed in Exercise 2 is included below:

In [ ]:
val combined_average_tmax = heathrowAverageTmax.join(wickairportAverageTmax)
val correlation = Statistics.corr(combined_average_tmax.map(_._2._1),combined_average_tmax.map(_._2._2),"pearson")

The result from this code is the following output:

Since 0.8355 > 0 and is close to 1 this tells us that there is a strong positive correlation between the average max temperature in Heathrow and in Wick.